In [20]:
import tensorflow as tf
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from tqdm import tqdm
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import os
import importlib
# from importlib_metadata import version
# from skopt import BayesSearchCV
# from skopt.space import Categorical

# dir_r3 = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/out'
dir_ml = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/out'
randseed = 42
print("random seed: ", randseed)
np.random.seed(randseed)

def choose_data(dat, test_size, val_size):
    if dat == 'ml2':
        train = pd.read_csv(os.path.join(dir_ml, 'ml_train2.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        test = pd.read_csv(os.path.join(dir_ml, 'ml_test2.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        val = None  # Assuming no validation set for 'ml2'
    elif dat == 'ml':
        ml_full = pd.read_csv(os.path.join(dir_ml, 'ml-1m_full.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")

        # Get unique user IDs

        user_ids = ml_full['userId'].unique()
        item_ids = ml_full['songId'].unique()

        n_users = len(user_ids)
        n_items = len(item_ids)
        # Split user IDs for train, validation, and test sets
        train, test = train_test_split(ml_full, test_size=test_size, random_state=42)
        train, val = train_test_split(train, test_size=val_size/(1-test_size), random_state=42)

    else:
        print('Wrong data input')
        return None, None, None

    # Print the sizes of the datasets
    print(f"Train set size: {train.shape[0]} ratings")
    print(f"Validation set size: {val.shape[0] if val is not None else 0} ratings")
    print(f"Test set size: {test.shape[0]} ratings")
    print(ml_full.shape[0])
    print(train.shape[0] + val.shape[0] +  test.shape[0])

    return train, val, test, n_users, n_items

def load_confounders(dat, k):
    CAUSEFIT_DIR = f'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/exposure_output/{dat}_exp_k_{k}.csv'
    conf_df = pd.read_csv(CAUSEFIT_DIR, header=None)
    confounder_data = conf_df.to_numpy().T
    return confounder_data

def load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=False, test_size=0.1, val_size=0.1):
    train, val, test, n_users, n_items = choose_data(dat, test_size, val_size)

    if train is None or test is None:
        return None, None, None, None, None

    if include_validation:
        # Use the provided validation set
        vad = val
    else:
        # Merge train and validation sets if validation is not required, and sort by userId
        train = pd.concat([train, val]).sort_values(by='userId').reset_index(drop=True)
        vad = None

    def build_matrix(df):
        rows = []
        cols = []
        ratings = []
        for line in df.itertuples():
            rows.append(line[1])
            cols.append(line[2])
            ratings.append(line[3])
        return csr_matrix((ratings, (rows, cols)), shape=(n_users, n_items)).todok()

    train_matrix = build_matrix(train)
    test_matrix = build_matrix(test)
    vad_matrix = build_matrix(vad) if vad is not None else None

    print("Load data finished. Number of users:", n_users, "Number of items:", n_items)
    return train_matrix, test_matrix, vad_matrix, n_users, n_items


from bayes_opt import BayesianOptimization

def run_model(module_name, class_name, k, dat='ml', include_validation=False, use_confounder=False, use_exposure=False, test_size=0.1, val_size=0.1, hidden_neuron=500, learning_rate=0.001, reg_rate=0.1, epoch=20, batch_size=200, verbose=False, T=1, display_step=1000, save_path=None):
    train, test, vad, user, item = load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=include_validation, test_size=test_size, val_size=val_size)

    confounder_data = None
    exposure_data = None

    if use_confounder:
        # Load confounder data
        confounder_data = load_confounders(dat, k)
    if use_exposure:
        # Create exposure matrix
        exposure_data = (train > 0).astype(np.float32).todense().T

    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.compat.v1.Session(config=config) as sess:
        module = importlib.import_module(module_name)
        model_class = getattr(module, class_name)
        final_model = model_class(sess, user, item, learning_rate=learning_rate, reg_rate=reg_rate, epoch=epoch, batch_size=batch_size, verbose=verbose, T=T, display_step=display_step)

        final_model.build_network(hidden_neuron=hidden_neuron)
        
        if confounder_data is not None and exposure_data is not None:
            final_model.execute(train, vad, confounder_data, exposure_data)
        elif confounder_data is not None:
            final_model.execute(train, vad, confounder_data)
        else:
            final_model.execute(train, vad)

        if vad is not None:
            if confounder_data is not None and exposure_data is not None:
                rmse, mae = final_model.test(vad, confounder_data, exposure_data)
            elif confounder_data is not None:
                rmse, mae = final_model.test(vad, confounder_data)
            else:
                rmse, mae = final_model.test(vad)
        else:
            rmse, mae = None, None

    return rmse, mae



random seed:  42


In [25]:
def objective_urec1conf(learning_rate, reg_rate, hidden_neuron, k_idx):
    k_values = [1, 2, 5, 10, 20, 32, 50, 100]
    k = k_values[int(k_idx)]

    lr_values = [0.0001, 0.001, 0.01, 0.1]
    learning_rate = lr_values[int(learning_rate)]

    rr_values = [0.001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
    reg_rate = rr_values[int(reg_rate)]


    # Run your model here with the specified hyperparameters
    final_rmse, final_mae = run_model(
        'urec_1_conf', 'UAutoRec1conf', k,
        dat='ml', include_validation=True, use_confounder=True, use_exposure=False,
        hidden_neuron=int(hidden_neuron), learning_rate=learning_rate, reg_rate=reg_rate,
        batch_size=500, epoch=30
    )

    return -final_rmse  # Minimize RMSE

def objective_urec2confexp(learning_rate, reg_rate, hidden_neuron, k):
    params = {
        'learning_rate': learning_rate,
        'reg_rate': reg_rate,
        'hidden_neuron': int(hidden_neuron),
        'k': int(k)
    }

    final_rmse, _ = run_model('urec_2_confexp', 'UAutoRec2confexp', params['k'], dat='ml', include_validation=True, use_confounder=True, use_exposure=True, hidden_neuron=params['hidden_neuron'], learning_rate=params['learning_rate'], reg_rate=params['reg_rate'], batch_size=500, epoch=30, verbose=True)

    return -final_rmse

In [27]:
# Define the parameter space
pbounds = {
    'learning_rate': (0, 3),
    'reg_rate': (0, 7),
    'hidden_neuron': (10, 500),  # You can adjust this as needed
    'k_idx': (0, 7)  # Indexes for k values
}
# Optimize for UAutoRec1conf
optimizer_urec1conf = BayesianOptimization(
    f=objective_urec1conf,
    pbounds=pbounds,
    random_state=42
)

optimizer_urec1conf.maximize(init_points=5, n_iter=20)

best_params = optimizer_urec1conf.max['params']

best_params_decoded = {
    'learning_rate': [0.0001, 0.001, 0.01, 0.1][int(best_params['learning_rate'])],
    'reg_rate': [0.001, 0.001, 0.01, 0.1, 1, 10, 100, 1000][int(best_params['reg_rate'])],
    'hidden_neuron': int(best_params['hidden_neuron']),
    'k': [1, 2, 5, 10, 20, 32, 50, 100][int(best_params['k_idx'])]
}

print("Best parameters:", best_params_decoded)

#Training: 100%|██████████| 30/30 [02:27<00:00,  4.91s/epoch, Loss=4.74e+4, RMSE=0.893, MAE=0.705]
# | 1         | -0.8926   | 193.5     | 6.655     | 2.196     | 4.191     |

|   iter    |  target   | hidden... |   k_idx   | learni... | reg_rate  |
-------------------------------------------------------------------------
Train set size: 800169 ratings
Validation set size: 100022 ratings
Test set size: 100022 ratings
1000213
1000213
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder.
Train data processed shape: (3706, 6040)
Confounder data shape: (3706, 6040)


Training: 100%|██████████| 30/30 [02:27<00:00,  4.91s/epoch, Loss=4.74e+4, RMSE=0.893, MAE=0.705]


| 1         | -0.8926   | 193.5     | 6.655     | 2.196     | 4.191     |
Train set size: 800169 ratings
Validation set size: 100022 ratings
Test set size: 100022 ratings
1000213
1000213
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder.
Train data processed shape: (3706, 6040)
Confounder data shape: (3706, 6040)


Training: 100%|██████████| 30/30 [02:09<00:00,  4.30s/epoch, Loss=9.65e+4, RMSE=1.07, MAE=0.876]


| 2         | -1.069    | 86.45     | 1.092     | 0.1743    | 6.063     |
Train set size: 800169 ratings
Validation set size: 100022 ratings
Test set size: 100022 ratings
1000213
1000213
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder.
Train data processed shape: (3706, 6040)
Confounder data shape: (3706, 6040)


...

| 23        | -0.9073   | 194.9     | 6.88      | 2.612     | 5.223     |
Train set size: 800169 ratings
Validation set size: 100022 ratings
Test set size: 100022 ratings
1000213
1000213
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder.
Train data processed shape: (3706, 6040)
Confounder data shape: (3706, 6040)


Training: 100%|██████████| 30/30 [02:28<00:00,  4.94s/epoch, Loss=5.63e+4, RMSE=0.936, MAE=0.742]


| 24        | -0.9356   | 484.2     | 2.555     | 1.68      | 1.873     |
Train set size: 800169 ratings
Validation set size: 100022 ratings
Test set size: 100022 ratings
1000213
1000213
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder.
Train data processed shape: (3706, 6040)
Confounder data shape: (3706, 6040)


Training: 100%|██████████| 30/30 [02:08<00:00,  4.27s/epoch, Loss=4.9e+4, RMSE=0.895, MAE=0.71]  


| 25        | -0.8948   | 196.7     | 6.216     | 2.897     | 4.526     |
Best parameters: {'learning_rate': 0.01, 'reg_rate': 1, 'hidden_neuron': 193, 'k': 50}


In [30]:
def run_model(module_name, class_name, k, dat='ml', include_validation=False, use_confounder=False, use_exposure=False, test_size=0.1, val_size=0.1, hidden_neuron=500, learning_rate=0.001, reg_rate=0.1, epoch=20, batch_size=200, verbose=False, T=1, display_step=1000, save_path=None):
    train, test, vad, user, item = load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=include_validation, test_size=test_size, val_size=val_size)

    confounder_data = None
    exposure_data = None

    if use_confounder:
        # Load confounder data
        confounder_data = load_confounders(dat, k)
    if use_exposure:
        # Create exposure matrix
        exposure_data = (train > 0).astype(np.float32).todense().T
    
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.compat.v1.Session(config=config) as sess:
        # Dynamically import the module and create an instance of the chosen class
        module = importlib.import_module(module_name)
        model_class = getattr(module, class_name)
        final_model = model_class(sess, user, item, learning_rate=learning_rate, reg_rate=reg_rate, epoch=epoch, batch_size=batch_size, verbose=verbose, T=T, display_step=display_step)

        final_model.build_network(hidden_neuron=hidden_neuron)
        
        # Determine how many arguments to pass to the execute function
        if confounder_data is not None and exposure_data is not None:
            final_model.execute(train, test, confounder_data, exposure_data)
        elif confounder_data is not None:
            final_model.execute(train, test, confounder_data)
        else:
            final_model.execute(train, test)

        # Save the training and test RMSE values
        if save_path:
            class_folder = os.path.join(save_path, class_name.lower())
            os.makedirs(class_folder, exist_ok=True)

            train_loss_file = os.path.join(class_folder, f'TUNED_train_loss_bayesian_{dat}_k{k}.npy')
            test_rmse_file = os.path.join(class_folder, f'TUNED_test_rmse_bayesian_{dat}_k{k}.npy')


            np.save(train_loss_file, np.array(final_model.train_loss_history))
            np.save(test_rmse_file, np.array(final_model.test_rmse_history))


# Best parameters: {'learning_rate': 0.01, 'reg_rate': 1, 'hidden_neuron': 193, 'k': 50} this was the long one 30 epochs 20 it and 5 random init
# Best parameters for UAutoRec1conf in detailed form: {'learning_rate': 0.00654788977303255, 'reg_rate': 1, 'hidden_neuron': 100, 'k': 100} 
# Best parameters for UAutoRec1conf in detailed form: {'learning_rate': 0.000934753482, 'reg_rate': 1, 'hidden_neuron': 100, 'k': 50}

    
result_path = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/results'
run_model('urec_1_conf', 'UAutoRec1conf', best_params_decoded['k'], dat='ml', include_validation=False, use_confounder=True, use_exposure=False, 
          hidden_neuron= best_params_decoded['hidden_neuron'], 
          learning_rate= best_params_decoded['learning_rate'], 
          reg_rate=best_params_decoded['reg_rate'], 
          batch_size=500, epoch=80, save_path = result_path)




Train set size: 800169 ratings
Validation set size: 100022 ratings
Test set size: 100022 ratings
1000213
1000213
Load data finished. Number of users: 6040 Number of items: 3706
UAutoRec with Confounder.
Train data processed shape: (3706, 6040)
Confounder data shape: (3706, 6040)


Training: 100%|██████████| 80/80 [05:58<00:00,  4.48s/epoch, Loss=3.44e+4, RMSE=0.963, MAE=0.763]
